In [1]:
import pandas as pd
import numpy as np
import random
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playerindex
l=playerindex.PlayerIndex()

In [11]:
from nba_api.stats.static import players
data=players.get_players()
full_player=pd.DataFrame(data)
full_player=full_player[full_player['is_active']==True]

In [2]:
player=l.get_data_frames()[0]
player=player[['PERSON_ID','PLAYER_FIRST_NAME','PLAYER_LAST_NAME','TEAM_NAME']]

In [3]:
player

,PERSON_ID,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,TEAM_NAME
0,1630173,Precious,Achiuwa,Raptors
1,203500,Steven,Adams,Grizzlies
2,1628389,Bam,Adebayo,Heat
3,1630534,Ochai,Agbaji,Jazz
4,1630583,Santi,Aldama,Grizzlies
...,...,...,...,...
536,201152,Thaddeus,Young,Raptors
537,1629027,Trae,Young,Hawks
538,1630209,Omer,Yurtseven,Heat
539,203469,Cody,Zeller,Heat


In [12]:
full_player.rename(columns={'id':'PERSON_ID'},inplace=True)

## Score

In [20]:
regular_season_score=[]
for i in player['PERSON_ID'].tolist():
    regular_season_score.append(playercareerstats.PlayerCareerStats(player_id=i).get_data_frames()[1].loc[0,'PTS'])

SSLError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/playercareerstats?LeagueID=&PerMode=Totals&PlayerID=1630792 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [30]:
len(regular_season_score)

541

In [29]:
#Continue Retrieving
for i in range(455,541):
    id=player.loc[i,'PERSON_ID']
    regular_season_score.append(playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[1].loc[0,'PTS'])

In [31]:
player['regular_season_TotalScore']=regular_season_score

In [61]:
# [3] is the total playoff socre df
playoff_score=[]
for i in player['PERSON_ID'].tolist():
    df=playercareerstats.PlayerCareerStats(player_id=i).get_data_frames()[3]
    if(df.empty):
        playoff_score.append(0)
    else:
        playoff_score.append(df.loc[0,'PTS'])

In [62]:
len(playoff_score)

541

In [119]:
# Continue Retreiving if needed
'''
player.loc[54,'PERSON_ID']
for i in range(524,541):
    id=player.loc[i,'PERSON_ID']
    df=playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[3]
    if(df.empty):
        playoff_score.append(0)
    else:
        playoff_score.append(df.loc[0,'PTS'])
'''

In [63]:
player['playoff_TotalScore']=playoff_score

In [69]:
player['Total_Score']=player['regular_season_TotalScore']+player['playoff_score_total']

In [74]:
active_player_list=player.merge(full_player,how='right',on='PERSON_ID')
active_player_list=active_player_list[['PERSON_ID','full_name','first_name','last_name','TEAM_NAME','regular_season_TotalScore','playoff_TotalScore','Total_Score']]

In [75]:
active_player_list

,PERSON_ID,full_name,first_name,last_name,TEAM_NAME,regular_season_TotalScore,playoff_TotalScore,Total_Score
0,1630173,Precious Achiuwa,Precious,Achiuwa,Raptors,1476.0,68.0,1544.0
1,203500,Steven Adams,Steven,Adams,Grizzlies,6518.0,509.0,7027.0
2,1628389,Bam Adebayo,Bam,Adebayo,Heat,6146.0,1095.0,7241.0
3,1630534,Ochai Agbaji,Ochai,Agbaji,Jazz,467.0,0.0,467.0
4,1630583,Santi Aldama,Santi,Aldama,Grizzlies,828.0,39.0,867.0
...,...,...,...,...,...,...,...,...
530,201152,Thaddeus Young,Thaddeus,Young,Raptors,14088.0,542.0,14630.0
531,1629027,Trae Young,Trae,Young,Hawks,8990.0,713.0,9703.0
532,1630209,Omer Yurtseven,Omer,Yurtseven,Heat,339.0,29.0,368.0
533,203469,Cody Zeller,Cody,Zeller,Heat,4300.0,113.0,4413.0


In [81]:
#data random validation
def test(l):
    for i in l:
        id = active_player_list.loc[i,'PERSON_ID']
        df_reg=playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[1]
        #df_off=playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[3]
        if (active_player_list.loc[i,'regular_season_TotalScore'] != df_reg.loc[0,'PTS']):
            return False
    return True

In [84]:
random_list = random.sample(range(1, 501), 5)
test(random_list)

True

## Salary

In [126]:
pg=pd.read_csv('src/PG.csv')
sg=pd.read_csv('src/SG.csv')
sf=pd.read_csv('src/SF.csv')
pf=pd.read_csv('src/PF.csv')
c=pd.read_csv('src/C.csv')
active_player_salary=pd.concat([pg,sg,sf,pf,c])


In [128]:
active_player_salary.dropna(how='any')

,Unnamed: 0,Unnamed: 1
8,Chris Paul,"$389,909,419"
9,Stephen Curry,"$354,721,977"
10,Russell Westbrook,"$340,265,800"
11,Damian Lillard,"$279,348,442"
12,Kyle Lowry,"$275,048,700"
...,...,...
420,Sandro Mamukelashvili,"$2,377,423"
425,Orlando Robinson,"$1,801,769"
430,Luka Garza,"$925,258"
435,Micah Potter,"$53,176"


In [86]:
active_player_salary.rename(columns={'Unnamed: 0':'full_name','Unnamed: 1':'Salary'},inplace=True)

In [100]:
money=active_player_salary['Salary'].str.replace('$','')
money=money.str.replace(',','')
money.tolist()

/var/folders/t4/g2w87qw154j05sg7m8hh2nx00000gn/T/ipykernel_45132/3582552877.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  money=active_player_salary['Salary'].str.replace('$','')


['389909419',
 '354721977',
 '340265800',
 '279348442',
 '275048700',
 '274127986',
 '270083674',
 '254083552',
 '166120847',
 '163018545',
 '155945859',
 '154378483',
 '149275194',
 '136469709',
 '132178909',
 '122254173',
 '115228300',
 '112353436',
 '110032562',
 '109148261',
 '108917826',
 '103296369',
 '102337423',
 '91531317',
 '89978794',
 '86790942',
 '85365412',
 '82027846',
 '81520846',
 '81018675',
 '74379608',
 '73625090',
 '71214269',
 '63088565',
 '61409999',
 '60104215',
 '58842940',
 '53702301',
 '49676633',
 '47556675',
 '41738951',
 '38630624',
 '35596275',
 '35552881',
 '34910931',
 '32337054',
 '31658280',
 '31101096',
 '30824951',
 '24739538',
 '24131515',
 '23869193',
 '23360347',
 '22414388',
 '21059090',
 '17878995',
 '17188836',
 '15887971',
 '14792140',
 '14500440',
 '14094745',
 '14034855',
 '13523560',
 '13246897',
 '12152560',
 '11292720',
 '10803948',
 '10449718',
 '10017385',
 '9770880',
 '9138840',
 '8970258',
 '8809320',
 '7723000',
 '6604800',
 '648000

In [101]:
active_player_salary['int_value']=money

In [102]:
active_player_salary

,full_name,Salary,int_value
8,Chris Paul,"$389,909,419",389909419
9,Stephen Curry,"$354,721,977",354721977
10,Russell Westbrook,"$340,265,800",340265800
11,Damian Lillard,"$279,348,442",279348442
12,Kyle Lowry,"$275,048,700",275048700
...,...,...,...
420,Sandro Mamukelashvili,"$2,377,423",2377423
425,Orlando Robinson,"$1,801,769",1801769
430,Luka Garza,"$925,258",925258
435,Micah Potter,"$53,176",53176


In [107]:
active_player_salary.to_csv('Salary.csv',index=False)

In [108]:
active_player_list.to_csv('Player.csv',index=False)

In [112]:
merged=active_player_list.merge(active_player_salary,how='right',on='full_name')

In [114]:
merged.dropna(inplace=True)

In [115]:
merged['PERSON_ID'].tolist()

[101108.0,
 201939.0,
 201566.0,
 203081.0,
 200768.0,
 201144.0,
 202681.0,
 201950.0,
 201565.0,
 1627732.0,
 1626156.0,
 201609.0,
 202704.0,
 1627750.0,
 201937.0,
 1627832.0,
 1628368.0,
 201588.0,
 1627763.0,
 1629029.0,
 1626179.0,
 1629027.0,
 203935.0,
 1628365.0,
 203915.0,
 201976.0,
 201988.0,
 1628983.0,
 202709.0,
 1629630.0,
 1627749.0,
 1629636.0,
 1626145.0,
 1628973.0,
 1628401.0,
 1629012.0,
 1626153.0,
 1628984.0,
 1630163.0,
 204456.0,
 1629632.0,
 1628420.0,
 1630595.0,
 1626166.0,
 1627739.0,
 1628373.0,
 1630165.0,
 1629645.0,
 1628372.0,
 1628975.0,
 1627780.0,
 1630169.0,
 1630184.0,
 1630175.0,
 1628988.0,
 1630558.0,
 1630200.0,
 1629216.0,
 203526.0,
 1629134.0,
 1630178.0,
 1630700.0,
 1630193.0,
 1630202.0,
 1630201.0,
 1630544.0,
 1630245.0,
 1629162.0,
 1630531.0,
 1630631.0,
 1631102.0,
 1630625.0,
 1630554.0,
 1629614.0,
 1630249.0,
 1630540.0,
 1630240.0,
 1630613.0,
 1630692.0,
 1630215.0,
 1630637.0,
 201935.0,
 202331.0,
 203078.0,
 202691.0,
 201

In [116]:
null_list=[]
for i in active_player_list['PERSON_ID']:
    if i not in merged['PERSON_ID'].tolist():
        null_list.append(i)